Imports

In [86]:
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import pprint
from gspread_dataframe import get_as_dataframe
import time
from datetime import date
import gspread
import datetime
import plotly.express as px


Connect to Google Sheets

In [87]:
service_account=gspread.service_account(filename="gambling-7-24-bf73ca0313d0.json")

In [88]:
sheet=service_account.open("Discipline")

In [89]:
worksheet=sheet.worksheet("Picks")

In [90]:
df = pd.DataFrame(worksheet.get_all_records())

In [91]:
df

,Date,Week,Sport,League,Gender,Timing,Type_of_Wager,Twitter,Odds,Outcome,Game_Details,Read,Bet,Win/Lose,% of Bankroll
0,8/8/2022,32,Soccer,Italy,Male,Full,Spread,erik,-110,W,Roma -.75,3,$13.00,$5.00,2.10%
1,8/8/2022,32,Soccer,Germany,Male,Full,Spread,adam,-110,L,Leverkusen -1.5,1,$12.50,-$12.50,2.02%
2,8/8/2022,32,Football,NFL,Male,Full,Total,adam,-110,L,Cardinals Bengals under 32,2,$11.00,-$11.00,1.78%
3,8/8/2022,32,Football,NFL,Male,Full,Spread,adam,-110,L,Eagles +2,2,$11.00,-$11.00,1.78%
4,8/8/2022,32,Tennis,ATP,Female,Live,Total,erik,-118,W,Vandewedghe Tauson over 28.5,2,$11.00,$10.00,1.78%
5,8/8/2022,32,Football,NFL,Male,Full,Total,adam,-110,L,Ravens Titans under 31,2,$11.00,-$11.00,1.78%
6,8/8/2022,32,Baseball,MLB,Male,Full,Total,erik,100,W,Cubs Reds under 9,3,$12.50,$10.00,2.02%
7,8/8/2022,32,Basketball,WNBA,Female,Full,Total,erik,-110,W,Liberty Wings over 163,3,$11.00,$10.00,1.78%
8,8/8/2022,32,Baseball,MLB,Male,Full,Total,gunit,100,W,Cubs Nationals under 7.5,4,$11.00,$10.00,1.78%
9,8/8/2022,32,Baseball,MLB,Male,Full,Total,gunit,100,W,Pirates DBacks over 8.5,3,$11.50,$10.00,1.86%


Dataframe Calculations

###### Running Total

In [92]:
df['Win/Lose']=df["Win/Lose"].replace({'\$':''}, regex = True)
df['Win/Lose']=pd.to_numeric(df["Win/Lose"])
running_total=df['Win/Lose'].sum()

###### Week Total

In [93]:
t1 = datetime.datetime.now()
current_week=t1.strftime("%U")
df['Week']=df['Week'].astype(str)
weekly_sum=df[df.Week==current_week]['Win/Lose'].sum()

Visualizations

In [94]:
df.head()

,Date,Week,Sport,League,Gender,Timing,Type_of_Wager,Twitter,Odds,Outcome,Game_Details,Read,Bet,Win/Lose,% of Bankroll
0,8/8/2022,32,Soccer,Italy,Male,Full,Spread,erik,-110,W,Roma -.75,3,$13.00,5.0,2.10%
1,8/8/2022,32,Soccer,Germany,Male,Full,Spread,adam,-110,L,Leverkusen -1.5,1,$12.50,-12.5,2.02%
2,8/8/2022,32,Football,NFL,Male,Full,Total,adam,-110,L,Cardinals Bengals under 32,2,$11.00,-11.0,1.78%
3,8/8/2022,32,Football,NFL,Male,Full,Spread,adam,-110,L,Eagles +2,2,$11.00,-11.0,1.78%
4,8/8/2022,32,Tennis,ATP,Female,Live,Total,erik,-118,W,Vandewedghe Tauson over 28.5,2,$11.00,10.0,1.78%


In [114]:
sport = px.histogram(df, 
            x="Sport", 
            y="Win/Lose", 
            color="Gender", 
            barmode="group",
            title=f"Historical Record by Sport/Gender"
            ,histfunc="sum"
            ,text_auto=True)

sport.update_layout(yaxis_title="Dollars ($)")
sport.update_xaxes(type='category')
sport.update_xaxes(categoryorder='total descending')
sport.update_layout(xaxis={'categoryorder':'total descending'}) # add only this line

sport.show()

In [113]:
week = px.histogram(df, 
            x="Week", 
            y="Win/Lose", 
            color="Sport", 
            barmode="group",
            title=f"Historical Record by Week/Sport"
            ,histfunc="sum"
            ,text_auto=True)

week.update_layout(yaxis_title="Dollars ($)")
week.update_xaxes(categoryorder='total descending')
week.update_yaxes(categoryorder='total descending')

week.show()

In [122]:
week = px.histogram(df, 
            x="Sport", 
            y="Win/Lose", 
            color="League", 
            barmode="group",
            title=f"Historical Record by Sport/League"
            ,histfunc="sum"
            ,text_auto=True)

week.update_layout(yaxis_title="Dollars ($)")
week.update_yaxes(categoryorder='total ascending')

week.show()